1. Instalar bibliotecas não-nativas

In [12]:
!pip install requests 
!pip install plyer


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\sthefp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\sthefp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


2. Importar bibliotecas

In [13]:
from plyer import notification 
from datetime import datetime
import pandas as pd 
import requests 

### ETL

- Base de dados "Corretoras"

In [14]:
base = "CORRETORAS"
etapa = "EXTRACAO"

# Extract
def corretoras_database():
    try:
        url1 = "https://brasilapi.com.br/api/cvm/corretoras/v1"
        response = requests.get(url1, timeout=3)

        if response.status_code == 200:
            data_corretoras = response.json()
            corretoras_full = pd.DataFrame(data_corretoras)
            return corretoras_full
    
        else:
            today = datetime.today()
            notification.notify(

                    title="ATENÇÃO: Alerta Baixo",

                    message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                    timeout=10 )   
    except requests.exceptions.Timeout:
        print("This request has timeout, try again later")

#Load
corretoras_database().to_excel("corretoras_database.xlsx", index = False)



- Base de dados "CNPJ"

In [16]:
base = "CNPJ"
etapa = "EXTRACAO"

#Extract

def cnpj_database():
    lista_cnpj = corretoras_database()['cnpj']
    cnpj_full = []

    for cnpj in lista_cnpj:
        try:
            url = 'https://brasilapi.com.br/api/cnpj/v1/{}'.format(cnpj)
            response = requests.get(url, timeout=3)
        except requests.exceptions.Timeout:
            continue
            
        if response.status_code == 200:
            data_cnpj = response.json()
            cnpj_full.append([data_cnpj['uf'], data_cnpj['cep'], data_cnpj['cnpj'],
                            data_cnpj['porte'],data_cnpj['numero'],data_cnpj['bairro'],
                            data_cnpj['ddd_fax'],data_cnpj['cnae_fiscal'],data_cnpj['complemento'],
                            data_cnpj['razao_social'],data_cnpj['nome_fantasia'],data_cnpj['ddd_telefone_1'],
                            data_cnpj['ddd_telefone_2'],data_cnpj['opcao_pelo_mei'],data_cnpj['codigo_municipio'],
                            data_cnpj['natureza_juridica'],data_cnpj['data_opcao_pelo_mei'],data_cnpj['data_exclusao_do_mei'],
                            data_cnpj['data_inicio_atividade'],data_cnpj['data_situacao_especial'],data_cnpj['opcao_pelo_simples'],
                            data_cnpj['data_opcao_pelo_simples'],data_cnpj['data_exclusao_do_simples'],data_cnpj['descricao_situacao_cadastral']])
        else:
            today = datetime.today()
            notification.notify(

                title="ATENÇÃO: Alerta Baixo",

                message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                timeout=10 )

# Transform
            
    cnpj_full = pd.DataFrame(cnpj_full, columns=["UF","CEP","CNPJ","Porte","Numero","Bairro","Fax","CNAE","Complemento","Razao Social",
                                                "Nome Fantasia","Telefone 1","Telefone 2","Opcao pelo MEI","Codigo Municipio","Natureza Juridica",
                                                "Data Opcao pelo MEI","Data Exclusao MEI","Data Inicio da Atividade","Data Situacao Especial",
                                                "Opcao pelo Simples","Data Opcao pelo Simples","Data Exclusao do Simples","Descricao Situacao Cadastral"])
    return cnpj_full

# Load 

cnpj_database().to_excel("cnpj_database.xlsx", index = False)


- Base de dados "CEP"

In [17]:

base = "CEP"
etapa = "EXTRACAO"

#Extract

def cep_database():
    lista_cep = corretoras_database()['cep']
    cep_full = []

    for cep in lista_cep:
        try:
            url = 'https://brasilapi.com.br/api/cep/v2/{}'.format(cep)
            response = requests.get(url, timeout=3)
            
        except requests.exceptions.Timeout:
            continue    

        if response.status_code == 200:
            data_cep = response.json()
            cep_full.append([data_cep['cep'], data_cep['state'], data_cep['city'], data_cep['street']])
        else:
            today = datetime.today()
            notification.notify(

                title="ATENÇÃO: Alerta Baixo",

                message=f"Falha no carregamento da base {base} na etapa {etapa}\n{today}",

                timeout=10 )

        
# Transform
            
    cep_full = pd.DataFrame(cep_full, columns=["CEP","UF","Cidade","Logradouro"])
    return cep_full

# Load

cep_database().to_excel("cep_database.xlsx", index = False)